In [185]:
import pandas as pd

In [186]:
stocksCollection = pd.read_json('data/vietStocks.json')

In [187]:
# Filter all stocks with eps > 0 or eps4Q > 0
stocksCollection = stocksCollection[(stocksCollection['market_cap'] > 100) & (stocksCollection['pe'] > 0) ]
stocksCollection.count()

_id                   978
accumylated_vol       978
avg_trading_vol_10    978
avg_trading_vol_5     978
book_value            978
ceiling_price         978
change_percent        978
change_price          978
company_name          978
date                  978
eps                   978
eps4Q                 978
ev_per_ebit           978
ev_per_ebitda         978
floor                 978
floor_price           978
icb_group             978
id                    978
index_code            978
market_cap            978
match_price           978
match_price_10        978
match_price_5         978
pb                    978
pb4Q                  978
pe                    978
pe4Q                  978
profit_after_tax      978
roa                   978
roe                   978
symbol                978
the_beta              978
the_beta4Q            978
year                  978
dtype: int64

In [188]:
# Create a new column, EY, which is the inverse of PE
stocksCollection['EY'] = 1 / stocksCollection['pe']

In [189]:
# Create two columns: EYRank and ROERank. EYRank contains the ranking of each stock's EY, and ROERank contains the ranking of each stock's ROE
stocksCollection['EYRank'] = stocksCollection['EY'].rank(ascending=False)
stocksCollection['ROERank'] = stocksCollection['roe'].rank(ascending=False)
stocksCollection['finalRank'] = stocksCollection['EYRank'] + stocksCollection['ROERank']

In [190]:
stocksCollection

,_id,accumylated_vol,avg_trading_vol_10,avg_trading_vol_5,book_value,ceiling_price,change_percent,change_price,company_name,date,...,roa,roe,symbol,the_beta,the_beta4Q,year,EY,EYRank,ROERank,finalRank
66,{'$oid': '651034a433bdb847a5cec711'},0,0,0.0,11942,1.80,0.00,0.0,Công ty cổ phần Chứng khoán BOS,2023-09-24,...,2.94,2.97,ART,0.00,0.000000,2021,0.268817,46.0,729.0,775.0
80,{'$oid': '651034a433bdb847a5cec72d'},0,0,0.0,9427,2.10,0.00,0.0,Công ty Cổ phần Nông dược H.A.I,2023-09-24,...,1.02,1.56,HAI,0.00,0.042734,2022,0.088889,491.0,795.5,1286.5
93,{'$oid': '651034a433bdb847a5cec747'},0,0,362160.0,10400,1.95,0.00,0.0,Công ty Cổ phần Tập đoàn Tiến Bộ,2023-09-24,...,-0.08,-0.16,TTB,1.91,0.918641,2023,0.012461,896.0,901.0,1797.0
125,{'$oid': '651034a433bdb847a5cec787'},427150,800790,553540.0,854,3.30,-10.34,-0.3,Tổng Công ty Cổ phần Xây lắp Dầu khí Việt Nam,2023-09-24,...,-1.19,-23.39,PVX,-0.94,1.008250,2023,0.037908,770.0,974.0,1744.0
152,{'$oid': '651034a433bdb847a5cec7bd'},68050,125361,76240.0,5050,3.30,3.45,0.1,Công ty cổ phần Đầu tư Nhà đất Việt,2023-09-24,...,-21.44,-33.63,PVL,0.00,0.000000,2023,0.000565,971.0,977.0,1948.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1726,{'$oid': '651034a533bdb847a5ced409'},10,40,140.0,102378,216.60,0.00,0.0,Công ty cổ phần Thực phẩm Cholimex,2023-09-24,...,17.11,29.58,CMF,-0.03,0.537388,2022,0.145349,227.0,53.0,280.0
1727,{'$oid': '651034a533bdb847a5ced40b'},40,290,320.0,62561,206.60,0.00,0.0,Công ty Cổ phần Vinacafé Biên Hòa,2023-09-24,...,18.00,22.15,VCF,0.08,0.207063,2023,0.058275,679.5,130.0,809.5
1729,{'$oid': '651034a533bdb847a5ced40f'},1820,18806,8720.0,86685,225.90,-1.07,-2.2,Công ty Cổ phần Mía đường Sơn La,2023-09-24,...,41.68,56.15,SLS,0.18,0.099415,2023,0.195312,108.5,10.0,118.5
1730,{'$oid': '651034a533bdb847a5ced411'},0,80,0.0,121271,256.40,0.00,0.0,Công ty Cổ phần Bia và Nước giải khát Hạ Long,2023-09-24,...,24.44,38.57,HLB,-0.07,0.000000,2022,0.184843,124.0,22.0,146.0


In [191]:
# Get the top 50 stocks with the highest finalRank
stocksCollection = stocksCollection.sort_values(by=['finalRank'], ascending=True).head(50)

In [192]:
# Get the symbol, company_name, EY, ROE, EYRank, and ROERank of the top 50 stocks
stocksCollection = stocksCollection[['symbol', 'company_name', 'EY', 'roe', 'EYRank', 'ROERank', 'finalRank']]
stocksCollection


,symbol,company_name,EY,roe,EYRank,ROERank,finalRank
997,ABC,Công ty cổ phần Truyền thông VMG,2.127660,245.91,1.0,1.0,2.0
966,TBH,Công ty Cổ phần Tổng Bách Hóa,0.632911,152.64,4.0,3.0,7.0
1707,PAT,CTCP Phốt Pho Apatit Việt Nam,0.490196,77.95,6.0,7.0,13.0
1541,TMB,Công ty Cổ phần Kinh doanh Than miền Bắc - Vin...,0.480769,61.28,7.0,9.0,16.0
1604,ICN,Công ty Cổ phần Đầu Tư Xây Dựng Dầu Khí IDICO,0.347222,146.94,19.0,4.0,23.0
805,MDC,Công ty Cổ phần Than Mông Dương - Vinacomin,0.400000,42.77,12.5,17.0,29.5
1222,CNT,Công ty Cổ phần Xây dựng và Kinh doanh vật tư,0.354610,43.80,15.0,15.0,30.0
1103,CST,Công ty cổ phần Than Cao Sơn - TKV,0.400000,35.25,12.5,30.0,42.5
1402,BDG,CÔNG TY CỔ PHẦN MAY MẶC BÌNH DƯƠNG,0.320513,39.07,24.0,20.0,44.0
1544,FT1,Công ty cổ phần Phụ tùng máy số 1,0.278552,51.70,40.0,11.0,51.0
